<br>
<font size='6'><b>Python with Arduino</b></font><br><br>

<table style="border-style: hidden; border-collapse: collapse;" width = "80%"> 
    <tr style="border-style: hidden; border-collapse: collapse;">
        <td width = 50% style="border-style: hidden; border-collapse: collapse;">

        </td>
        <td width = 30%>
        by Seungchul Lee<br>
        iSystems Design Lab.<br>http://isystems.unist.ac.kr/<br>UNIST
        </td>
    </tr>
</table>

Table of Contents
<div id="toc"></div>

# 0. Installation
<br>
__ANACONDA Install Confirmation__
<br>

Before getting started with serial communication with Python, you must make sure pip is installed. However, pip will be automatically installed if ANACONDA is installed. Thus, you will have to do this step _only if you have not done it last time_. Below is the link for the instructions given last time.


http://nbviewer.jupyter.org/github/i-systems/IoT/blob/master/Mechatronics%20Class/HW/Python%20Installation%20Instructions.ipynb

<br>

__pip pyserial Installation__


<img src = "./image_files/command screen.png" width = 350>

Open the cmd window (press the windows key + r and type 'cmd'), and type in the following:
<br>
```c
python -m pip install pyserial
```

<img src = "./image_files/cmd screen.png" width = 550>


The download process will begin soon, and the pyserial module will be successively installed.

# 1. LED with python


<br>
Starting from this section, we will be using Python along with Arduino. As mentioned in previous classes, the codes being uploaded to the Arduino may have some limitations regarding the functionality. Starting from simple tasks such as byte sending through serial communication, we will be able to plot real time graphs based on the streamed data from Arduino.

<br>
$$
\large \text{Python in PC} \quad \overset{\text{Serial}}{\longleftrightarrow} \quad \text{Arduino} \quad \overset{\text{Digital Output}}{\longleftrightarrow} \quad \text{LED}$$

<br>

<img src = "./image_files/Python_wArduino_LED_fritz.png" width = 350>

<font size='4'><b>Lab 1: LED ON/OFF - Transmit (numeric) data from Python</b></font>

__Arduino code__

```c
int ON = 0;
const int pin = 9;

void setup() {
  pinMode(pin, OUTPUT);
  Serial.begin(9600);
}

void loop() {    
  if (Serial.available() > 0) {
    ON = Serial.read();
    
    if (ON == 1)
      digitalWrite(pin, HIGH);
    else if (ON == 0)
      digitalWrite(pin, LOW);

    Serial.println(ON);
  }
}
```

__Python code__

Before running this section, please make sure you are using the correct port. The sample code will be using 'COM4'.

In [2]:
import serial
import time

ser = serial.Serial('COM4', 9600, timeout=1)

In [ ]:
ser.write(bytes([1]))

In [ ]:
ser.write(bytes([0]))

You have to close ser to disconnect. Otherwise other devices cannot use this serial communication. 

In [8]:
ser.close()

Now we are ready to combine all into python with serial communication to Arduino to control LEDs.

In [9]:
ser = serial.Serial('COM4', 9600, timeout=1)

for i in range(10):
    ser.write(bytes([1]))
    time.sleep(0.5)
    ser.write(bytes([0]))
    time.sleep(0.5)

ser.close()    

<font size='4'><b>Lab 2: LED ON/OFF - Transmit (string) data from Python</b></font>

__arduino__

```c
String LEDcmd = "";
const int pin = 9;

void setup() {
  pinMode(pin, OUTPUT);
  Serial.begin(9600);
  Serial.setTimeout(50);
}

void loop() {
  if (Serial.available() > 0) {
    LEDcmd = Serial.readStringUntil('\n');

    if (LEDcmd == "ON")
      digitalWrite(pin, HIGH);
    else if (LEDcmd == "OFF")
      digitalWrite(pin, LOW);
  }
}
```

In [ ]:
str('ON').encode('UTF-8')

In [22]:
ser = serial.Serial('COM4', 9600, timeout=1)

In [ ]:
ser.write(str('ON').encode('UTF-8'))

In [ ]:
ser.write(str('OFF').encode('UTF-8'))

In [25]:
ser.close()

In [ ]:
ser = serial.Serial('COM14', 9600, timeout=1)

for i in range(10):
    ser.write(str('ON').encode('UTF-8'))
    time.sleep(0.5)
    ser.write(str('OFF').encode('UTF-8'))
    time.sleep(0.5)

ser.close()

<font size='4'><b>Lab 3: LED Brightness</b></font>

__arduino__

```c
int brightness = 0;
const int pin = 9;

void setup() {
  pinMode(pin, OUTPUT);
  Serial.begin(9600);
  Serial.setTimeout(50);
}

void loop() {
  if (Serial.available() > 0) {
    brightness = Serial.read();
    analogWrite(pin, brightness);
  }
}
```
<br>
__python code__

In [11]:
ser = serial.Serial('COM18', 9600, timeout=1)

for i in range(255):
    ser.write(bytes([i]))
    time.sleep(0.1)

for i in range(255):
    ser.write(bytes([255-i])) 
    time.sleep(0.1)    
    
ser.close() 

<font size='4'><b>Lab 4: Current Time on LCD Display</b></font>

<img src = "./image_files/lcd_new_scheme.png" width = 500>

__arduino__

```c
#include <LiquidCrystal.h>

LiquidCrystal lcd(12, 11, 5, 4, 3, 2);
String LCDRead = "";

void setup() {
  // set up the LCD's number of columns and rows:
  lcd.begin(16, 2);
  lcd.setCursor(0,0);
  // Print a message to the LCD.
  lcd.print("The current time");
  Serial.begin(9600);
}

void loop() {
  if (Serial.available()) {

    LCDRead = Serial.readStringUntil('\n');

    lcd.setCursor(0, 1);
    lcd.print(LCDRead);
    
  }

}
```

__Displaying current time once__

In [ ]:
import serial
import time

ser = serial.Serial('COM18', 9600, timeout=1)

In [ ]:
print(datetime.now().strftime('%H:%M:%S'))
ser.write((datetime.now().strftime('%H:%M:%S') + '\n').encode('UTF-8'))

In [ ]:
ser.close()

__Displaying the time for 15 seconds__

In [ ]:
import serial
import time

ser = serial.Serial('COM18', 9600, timeout=1)

from datetime import datetime
for i in range(15):
    print(datetime.now().strftime('%H:%M:%S').encode('UTF-8'))
    ser.write((datetime.now().strftime('%H:%M:%S') + '\n').encode('UTF-8'))
    time.sleep(1)
    
ser.close()

# 2. Servo Motor with python

<br>
$$
\large \text{Python in PC} \quad \overset{\text{serial}}{\longleftrightarrow} \quad \text{arduino} \quad \overset{\text{PWM}}{\longleftrightarrow} \quad \text{servo}$$

<br>
<img src = "./image_files/servo.jpg" width = 400>

__Arduino code__

```c
#include <Servo.h>

Servo myservo;
cost int pint = 9;
int pos = 0;

void setup() {
  myservo.attach(pin);
  myservo.write(0);
  Serial.begin(9600);
}

void loop() {
  if (Serial.available() > 0) {
    pos = Serial.read();
    myservo.write(pos);
    delay(50);
  }
}
```

__Python code__

In [6]:
ser = serial.Serial('COM18', 9600, timeout=1)

In [8]:
ser.write(bytes([90]))

1

In [7]:
ser.write(bytes([180]))

1

In [9]:
ser.close()

In [11]:
ser = serial.Serial('COM18', 9600, timeout=1)

for angle in range(180):
    ser.write(bytes([angle]))
    time.sleep(0.01)
    
for angle in range(180):
    ser.write(bytes([180 - angle]))
    time.sleep(0.01)
    
ser.close()

In [12]:
import serial
import time

def move(angle):
    if (0 <= angle <= 180):
        ser.write(bytes([angle]))
    else:
        print("Servo angle must be an integer between 30 and 180.\n")

# Start the serial port to communicate with arduino
ser = serial.Serial('COM18', 9600, timeout=1)

print("The initial servo angle is 30, type 'end' if you want to stop")

while 1:
    angle = input("Enter your angle: ")
    if angle == "end":
        print("Finished")
        ser.close()
        break
    else:
        move(int(angle))
        time.sleep(0.01)
        print("The current servo angle is " + angle)


The initial servo angle is 30, type 'end' if you want to stop
Enter your angle: 180
The current servo angle is 180
Enter your angle: 20
The current servo angle is 20
Enter your angle: end
Finished


In [103]:
ser.close()

In [ ]:
import serial

In [ ]:
port = serial.Serial('COM18',9600, timeout = 1)

In [ ]:
line = port.readline()
print(line)

In [ ]:
port.close()

In [ ]:
port = serial.Serial('COM18',9600, timeout = 1)

while (port.inWaiting() == 0):
    print(port.readline())

In [ ]:
port.close()

In [ ]:
import numpy as np
port = serial.Serial('COM18',9600, timeout = 1)

val = []
for i in range(10):
    tmp = port.readline().strip()
    print(tmp)
    print(float(tmp))
    #val.append(float(tmp))
    
    #print port.readline()
    #val.append()
port.close()    
#print val

In [ ]:
ndval = np.array(val)
fval = ndval.astype(np.float)
print ndval
#print fval

In [ ]:
b = [];
for i in range(10):
    b.append(a[i])

float(b)    

In [ ]:
a = np.asarray(val)
print a

In [ ]:
print float(val[1])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(a)
plt.show()

# 3. Data Streaming with python




arduino -> python (IMU, matplotlib)

http://www.toptechboy.com/using-python-with-arduino-lessons/





## 3.1 MPU6050 and layout

Reference: http://playground.arduino.cc/Main/MPU-6050

The InvenSense MPU-6050 sensor contains a MEMS accelerometer and a MEMS gyro in a single chip. It is very accurate, as it contains 16-bits analog to digital conversion hardware for each channel. Therefor it captures the x, y, and z channel at the same time. The sensor uses the I2C-bus to interface with the Arduino.

<table style="border-style: hidden; border-collapse: collapse;" width = "90%"> 
    <tr style="border-style: hidden; border-collapse: collapse;">
        <td width = 25% style="border-style: hidden; border-collapse: collapse;">
<img src = "./image_files/mpu6050.jpg" width = 150>
        </td>
        <td width = 65%>
<img src = "./image_files/circuit.png" width = 600>
        </td>
    </tr>
</table>





__Arduino code__

Reference: http://playground.arduino.cc/Main/MPU-6050#short

<br>
```c
#include<Wire.h>

const int MPU_addr = 0x68;              // I2C address of the MPU-6050
int16_t AcX, AcY, AcZ, Tmp, GyX, GyY, GyZ;
void setup() {
  Wire.begin();
  Wire.beginTransmission(MPU_addr);
  Wire.write(0x6B);                     // PWR_MGMT_1 register
  Wire.write(0);                        // set to zero (wakes up the MPU-6050)
  Wire.endTransmission(true);
  Serial.begin(115200);
}

void loop() {
  Wire.beginTransmission(MPU_addr);
  Wire.write(0x3B);                     // starting with register 0x3B (ACCEL_XOUT_H)
  Wire.endTransmission(false);
  Wire.requestFrom(MPU_addr, 14, true); // request a total of 14 registers

  AcX = Wire.read() << 8 | Wire.read(); // 0x3B (ACCEL_XOUT_H) & 0x3C (ACCEL_XOUT_L)
  AcY = Wire.read() << 8 | Wire.read(); // 0x3D (ACCEL_YOUT_H) & 0x3E (ACCEL_YOUT_L)
  AcZ = Wire.read() << 8 | Wire.read(); // 0x3F (ACCEL_ZOUT_H) & 0x40 (ACCEL_ZOUT_L)
  Tmp = Wire.read() << 8 | Wire.read(); // 0x41 (TEMP_OUT_H) & 0x42 (TEMP_OUT_L)
  GyX = Wire.read() << 8 | Wire.read(); // 0x43 (GYRO_XOUT_H) & 0x44 (GYRO_XOUT_L)
  GyY = Wire.read() << 8 | Wire.read(); // 0x45 (GYRO_YOUT_H) & 0x46 (GYRO_YOUT_L)
  GyZ = Wire.read() << 8 | Wire.read(); // 0x47 (GYRO_ZOUT_H) & 0x48 (GYRO_ZOUT_L)

  Serial.print(AcX);
  Serial.print(',');
  Serial.print(AcY);
  Serial.print(',');
  Serial.println(AcZ);
  Serial.print(',');
  Serial.print(GyX);
  Serial.print(',');
  Serial.print(GyY);
  Serial.print(',');
  Serial.println(GyZ);
  delay(100);
}
```

__Python code__

reference: http://www.toptechboy.com/tutorial/python-with-arduino-lesson-11-plotting-and-graphing-live-data-from-arduino-with-matplotlib/

```python
import serial # import Serial Library
import numpy as np# Import numpy
import matplotlib.pyplot as plt 
%matplotlib qt 
from drawnow import *

# Define Variable
Accx = []
Accy = []
Accz = []

arduinoData = serial.Serial('com15',115200)     # Creating our serial object named arduinoData

plt.ion()   # Tell matplotlib you want interactive mode to plot live data

cnt = 0     # count

def makeFig():      # Create a function that makes our desired plot
    # plt.ylim(80,90)
    plt.figure(1)
    plt.subplot(311)                    # Set y min and max values
    plt.title('Streaming Data')         # Plot the title
    plt.grid(True)                      # Turn the grid on
    plt.ylabel('Acceleration')          # Set ylabels
    plt.plot(Accx, 'ro-', label='X')    # plot the temperature
    plt.legend(loc='upper left')        # plot the legend
    
    plt.subplot(312)                    # Set y min and max values
    plt.grid(True)                      # Turn the grid on
    plt.ylabel('Acceleration')          # Set ylabels
    plt.plot(Accy, 'ro-', label='Y')    # plot the temperature
    plt.legend(loc='upper left')          
    
    plt.subplot(313)                    # Set y min and max values
    plt.grid(True)                      # Turn the grid on
    plt.ylabel('Acceleration')          # Set ylabels
    plt.plot(Accz, 'ro-', label='Z')    # plot the temperature
    plt.legend(loc='upper left')          

    plt.show()


while True: # While loop that loops forever
    while (arduinoData.inWaiting() == 0):   # Wait here until there is data
        pass                                # do nothing
        
    arduinoString = arduinoData.readline().decode("utf-8") #.strip()
    # print(arduinoString)#read the line of text from the serial port
    dataArray = (arduinoString.split(','))
    # print(dataArray)
    # dataArray = np.array(arduinoString)
    # print(dataArray)
    # dataArray = arduinoString.split(",")  # Split it into an array called dataArray
    temp1 = float(dataArray[0])             # Convert first element to floating number and put in temp
    temp2 = float(dataArray[1]) 
    temp3 = float(dataArray[2])
   
    # Convert second element to floating number and put in P
    Accx.append(temp1)                      # Build our tempF array by appending temp readings
    Accy.append(temp2)                      # Building our pressure array by appending P readings
    Accz.append(temp3)
    
    drawnow(makeFig)    # Call drawnow to update our live graph
                      
    plt.pause(.000001)  # Pause Briefly. Important to keep drawnow from crashing
    cnt = cnt + 1

    if(cnt > 50):                           # If you have 50 or more points, delete the first one from the array
        Accx.pop(0)                         # This allows us to just see the last 50 data points
        Accy.pop(0)
        Accz.pop(0)
```        

<img src = "image_files\live.png" style="border:1px solid black", width = 400>

# 4. Summary

In [3]:
%%html
<iframe src="https://www.youtube.com/embed/jemrZxfKVIY" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

In [2]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>